In [1]:
#dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import gmaps.datasets
from scipy.stats import linregress
from scipy import stats
import os

In [5]:
#load assesment file
Hills_Values = os.path.join ("../../GB/Resources/HillsboroughAsmt.csv")
#create dataframe
Hills_Values_df = pd.read_csv(Hills_Values, dtype={'PHY_ZIPCD':'str','TOT_LVG_AREA':'str','ACT_YR_BLT':'str'})
Hills_Values_df.head()

# crime_path = os.path.join ("../Resources/combined_crime_data.csv")
# crime_data = pd.read_csv(crime_path)

,CO_NO,SEQ_NO,PARCEL_ID,ASMNT_YR,JV,LND_VAL,LND_UNTS_CD,NO_LND_UNTS,LND_SQFOOT,IMP_QUAL,...,OWN_ZIPCD,S_LEGAL,NBRHD_CD,CENSUS_BK,PHY_ADDR1,PHY_ADDR2,PHY_CITY,PHY_ZIPCD,ALT_KEY,STATE_PAR_ID
0,39,1,153323ZZZ000000000200A,2020,4628604,4554224,1,38842.0,13634755.0,NaN,...,33711.0,23 24 25 AND 26-33-15,22700200,120579900000,EGMONT KEY,NaN,ST PETERSBURG,33715,10000,C39-000-383-0762-5
1,39,2,153323ZZZ000000000201A,2020,100,100,5,1.0,NaN,NaN,...,33602.0,TAMPA BAY,22700200,120579900000,NaN,NaN,NaN,NaN,2900000202,C39-001-120-8468-6
2,39,3,153323ZZZ000000000400A,2020,208044,142794,1,550.0,267803.0,NaN,...,33601.0,THAT PART OF GOVT LOT 1 DESC A,22700200,120579900000,NaN,NaN,ST PETERSBURG,33715,50000,C39-000-383-0763-3
3,39,4,172701001000000000001U,2020,147699,5875,1,250.0,108816.0,NaN,...,33556.0,KEYSTONE PARK COLONY,21100700,120570115091,19507 PINE TREE RD,NaN,ODESSA,33556,370275,C39-000-421-5629-9
4,39,5,172701001000000000010U,2020,140829,134439,1,458.0,205316.0,NaN,...,33556.0,KEYSTONE PARK COLONY,21100700,120570115092,19931 ANGEL LN,NaN,ODESSA,33556,80000,C39-000-421-5630-7


In [22]:
#pick necessary columns
Home_Values_df = Hills_Values_df [['SEQ_NO','PARCEL_ID','ASMNT_YR','JV','NBRHD_CD','CENSUS_BK','PHY_ADDR1','PHY_CITY','PHY_ZIPCD','TOT_LVG_AREA','ACT_YR_BLT']]
Home_Values_df.head()

,SEQ_NO,PARCEL_ID,ASMNT_YR,JV,NBRHD_CD,CENSUS_BK,PHY_ADDR1,PHY_CITY,PHY_ZIPCD,TOT_LVG_AREA,ACT_YR_BLT
0,1,153323ZZZ000000000200A,2020,4628604,22700200,120579900000,EGMONT KEY,ST PETERSBURG,33715,NaN,NaN
1,2,153323ZZZ000000000201A,2020,100,22700200,120579900000,NaN,NaN,NaN,NaN,NaN
2,3,153323ZZZ000000000400A,2020,208044,22700200,120579900000,NaN,ST PETERSBURG,33715,NaN,NaN
3,4,172701001000000000001U,2020,147699,21100700,120570115091,19507 PINE TREE RD,ODESSA,33556,NaN,NaN
4,5,172701001000000000010U,2020,140829,21100700,120570115092,19931 ANGEL LN,ODESSA,33556,NaN,NaN


In [23]:
#remove any null values 
home_value1_df =Home_Values_df.dropna()
home_value1_df.head()

,SEQ_NO,PARCEL_ID,ASMNT_YR,JV,NBRHD_CD,CENSUS_BK,PHY_ADDR1,PHY_CITY,PHY_ZIPCD,TOT_LVG_AREA,ACT_YR_BLT
5,6,172701001000000000011U,2020,462229,21100700,120570115092,19859 ANGEL LN,ODESSA,33556,3972,1996
6,7,172701001000000000012U,2020,496434,21100700,120570115091,7320 W COUNTY LINE RD,ODESSA,33556,4643,2003
7,8,172701001000000000013U,2020,358083,21100700,120570115091,7212 W COUNTY LINE RD,ODESSA,33556,3269,1999
8,9,172701001000000000014U,2020,819704,21100700,120570115091,7222 W COUNTY LINE RD,ODESSA,33556,8203,2008
9,10,172701001000000000015U,2020,325923,21100700,120570115091,7326 W COUNTY LINE RD,ODESSA,33556,2608,2003


In [24]:
#remame columns
renamed_df = home_value1_df.rename(columns={"JV": "PropertyValue",
                                            "PHY_ADDR1": "PropAddress",
                                            "ASMNT_YR": "AssmntYr",
                                            "PHY_CITY":"City",
                                            "PHY_ZIPCD":"ZipCode",
                                             "TOT_LVG_AREA":"BldgSqFt",
                                           "ACT_YR_BLT": "YearBuilt"})
renamed_df.head()

,SEQ_NO,PARCEL_ID,AssmntYr,PropertyValue,NBRHD_CD,CENSUS_BK,PropAddress,City,ZipCode,BldgSqFt,YearBuilt
5,6,172701001000000000011U,2020,462229,21100700,120570115092,19859 ANGEL LN,ODESSA,33556,3972,1996
6,7,172701001000000000012U,2020,496434,21100700,120570115091,7320 W COUNTY LINE RD,ODESSA,33556,4643,2003
7,8,172701001000000000013U,2020,358083,21100700,120570115091,7212 W COUNTY LINE RD,ODESSA,33556,3269,1999
8,9,172701001000000000014U,2020,819704,21100700,120570115091,7222 W COUNTY LINE RD,ODESSA,33556,8203,2008
9,10,172701001000000000015U,2020,325923,21100700,120570115091,7326 W COUNTY LINE RD,ODESSA,33556,2608,2003


In [20]:
#create data frame for specific zip code


Zip13_df = renamed_df.loc[(renamed_df["ZipCode"]=="33613")]
Zip56_df = renamed_df.loc[(renamed_df["ZipCode"]=="33556")]

# Zip13_df.head()
# Zip56_df.head()

In [29]:
# create dataframe with properties less than 2500 sqft
#the theory here is that most Fl homes are between 1000 and 2500sqft, this will help eliminate outliers that skew the average prop valueHills2020_df = Zip_df.loc[(Zip_df["BldgSqFt"]<"2500")
Hills13_df = Zip13_df.loc[(Zip13_df["BldgSqFt"] < "2500")]
Hills56_df = Zip56_df.loc[(Zip56_df["BldgSqFt"] < "2500")]
# Hills13_df.head()
# Hills56_df.head()

In [36]:
# Drop outlier indices

# Calculate the average property value by ZIP


home_average13_df = pd.DataFrame(Hills13_df["PropertyValue"].mean())

home_average13_df.head()

ValueError: DataFrame constructor not properly called!

In [ ]:
crime_data.head()

In [ ]:
# Add average property value to crime dataframe
for i, row in home_average_df.iterrows():
    crime_data.loc[crime_data['zip'] == row['ZipCode'], "Avg Prop Value"] = row['PropertyValue']

crime_data.head()

In [ ]:
crime_data = pd.DataFrame(crime_data,columns=["Reported Date", "Case Number", "Crime Type", "Crime Type Extension",
                                              "year", "zip"])
crime_data.head()

In [ ]:
index_crime_19 = crime_data[(crime_data['year'] != 2019)].index 
crime_data_19 = crime_data.drop(index_crime_19, inplace = False)

crime_data_19.head()

In [ ]:
#Create scatter plot w/ regression line for school grade vs. prop value

# Set x and y values for scatter plot
x_values = crime_data_19['Avg Prop Value']
y_values = crime_data_19['Crime Type']
# y_values = crime_data_19['Reported Date']
# y_values = crime_data_19['Case Number']
# y_values = crime_data_19['year']
# y_values = crime_data_19['zip']

# Perform a linear regression on school grade vs. average property value
(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)

# Get regression values
regress_values = x_values * slope + intercept

# Create line equation string
line_eq = "y = " + str(round(slope,2)) + "x +" + str(round(intercept,2))

# Create scatter plot and
# set plot specs
plt.scatter(x_values, y_values)
plt.plot(x_values,regress_values,"r-")
plt.xlabel('Average Property Value in School ZIP (by $)')
plt.ylabel("School Grade (% of total possible points)")
plt.ylim(0, 100)
plt.xticks(rotation=45)
plt.annotate(line_eq,(5,-40),fontsize=15,color="red")
plt.grid(which='major', axis='both')
plt.title("School Grade vs. Average Property Value")

# Print r value
print(f"The r- is: {rvalue**2}")

# Display plot
plt.show()